In [1]:
import numpy as np
import pandas as pd

import itertools
import geopandas as gpd
from shapely.geometry import Point
from geopandas.tools import geocode
from geopy.geocoders import Nominatim

In [2]:
# read in the dataset we want lats and longs from
df = pd.read_csv('../../Data/ICTO_Datasets/features_ICTO_Datasets.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,MinistryMonthlyBudgetAmount,StartDate,EndDate,Marital_Status,Number_of_Household_Members,City,State,PostalCode,Country,Coded_Marital_Status,End_Yr,Start_Yr,End_Mnth,Start_Mnth
0,0,1,5485,3/1/2013,12/31/2013,Married,2,Erlanger,KY,41018,United States of America,2.0,2013.0,2013.0,12.0,3.0
1,1,2,4454,11/1/2011,4/30/2012,Married,2,Spanish Fort,AL,36527,United States of America,2.0,2012.0,2011.0,4.0,11.0
2,2,3,767,1/1/2013,12/31/2013,Single,1,Lexington,KY,40517,United States of America,4.0,2013.0,2013.0,12.0,1.0
3,3,4,6368,7/1/2015,2/29/2016,Married,2,Manheim,PA,17545,United States of America,2.0,2016.0,2015.0,2.0,7.0
4,4,5,1919,8/1/2011,7/31/2012,Divorced,1,Jacksonville,FL,32258-5434,United States of America,1.0,2012.0,2011.0,7.0,8.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8900 entries, 0 to 8899
Data columns (total 16 columns):
Unnamed: 0                     8900 non-null int64
Unnamed: 0.1                   8900 non-null int64
MinistryMonthlyBudgetAmount    8900 non-null int64
StartDate                      8900 non-null object
EndDate                        8900 non-null object
Marital_Status                 8900 non-null object
Number_of_Household_Members    8900 non-null int64
City                           8900 non-null object
State                          8900 non-null object
PostalCode                     8900 non-null object
Country                        8900 non-null object
Coded_Marital_Status           8890 non-null float64
End_Yr                         8899 non-null float64
Start_Yr                       8899 non-null float64
End_Mnth                       8899 non-null float64
Start_Mnth                     8899 non-null float64
dtypes: float64(5), int64(4), object(7)
memory usage: 1.1+ M

In [4]:
df['Address'] = df['City'] + ', ' + df['State'] + ', ' + df['Country']
df.head()

,Unnamed: 0,Unnamed: 0.1,MinistryMonthlyBudgetAmount,StartDate,EndDate,Marital_Status,Number_of_Household_Members,City,State,PostalCode,Country,Coded_Marital_Status,End_Yr,Start_Yr,End_Mnth,Start_Mnth,Address
0,0,1,5485,3/1/2013,12/31/2013,Married,2,Erlanger,KY,41018,United States of America,2.0,2013.0,2013.0,12.0,3.0,"Erlanger, KY, United States of America"
1,1,2,4454,11/1/2011,4/30/2012,Married,2,Spanish Fort,AL,36527,United States of America,2.0,2012.0,2011.0,4.0,11.0,"Spanish Fort, AL, United States of America"
2,2,3,767,1/1/2013,12/31/2013,Single,1,Lexington,KY,40517,United States of America,4.0,2013.0,2013.0,12.0,1.0,"Lexington, KY, United States of America"
3,3,4,6368,7/1/2015,2/29/2016,Married,2,Manheim,PA,17545,United States of America,2.0,2016.0,2015.0,2.0,7.0,"Manheim, PA, United States of America"
4,4,5,1919,8/1/2011,7/31/2012,Divorced,1,Jacksonville,FL,32258-5434,United States of America,1.0,2012.0,2011.0,7.0,8.0,"Jacksonville, FL, United States of America"


In [5]:
# read in outside data for finding lats and longs
df_out = pd.read_csv('../../Data/OutsideData/us-zip-code-latitude-and-longitude.csv')
df_out.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,71937,Cove,AR,34.398483,-94.39398,-6,1,"34.398483, -94.39398"
1,72044,Edgemont,AR,35.624351,-92.16056,-6,1,"35.624351, -92.16056"
2,56171,Sherburn,MN,43.660847,-94.74357,-6,1,"43.660847, -94.74357"
3,49430,Lamont,MI,43.010337,-85.89754,-5,1,"43.010337, -85.89754"
4,52585,Richland,IA,41.194129,-91.98027,-6,1,"41.194129, -91.98027"


In [6]:
# create mini dataframe for geo coordinates
geo_coords = df_out['geopoint']
geo_coords

0         34.398483, -94.39398
1         35.624351, -92.16056
2         43.660847, -94.74357
3         43.010337, -85.89754
4         41.194129, -91.98027
                 ...          
43186     40.055411, -75.13793
43187     31.334062, -83.59971
43188     42.005815, -85.46428
43189      28.852564, -82.0321
43190    42.614852, -73.970812
Name: geopoint, Length: 43191, dtype: object

In [7]:
# split dataframe by comma
split_coords=geo_coords.str.split(',')

In [8]:
# create new mini data frame for lats and longs
sliced_coords = pd.DataFrame(geo_coords.str.split(',',1).tolist(),
                             columns = ['latitude','longitude'])
sliced_coords.head()

,latitude,longitude
0,34.398483,-94.39398
1,35.624351,-92.16056
2,43.660847,-94.74357
3,43.010337,-85.89754
4,41.194129,-91.98027


In [9]:
# concatenat the new columns latitude and longitude to the new dataframe
frames = [df_out, sliced_coords]
df_new = pd.concat(frames, axis=1)
df_new.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint,latitude,longitude
0,71937,Cove,AR,34.398483,-94.39398,-6,1,"34.398483, -94.39398",34.398483,-94.39398
1,72044,Edgemont,AR,35.624351,-92.16056,-6,1,"35.624351, -92.16056",35.624351,-92.16056
2,56171,Sherburn,MN,43.660847,-94.74357,-6,1,"43.660847, -94.74357",43.660847,-94.74357
3,49430,Lamont,MI,43.010337,-85.89754,-5,1,"43.010337, -85.89754",43.010337,-85.89754
4,52585,Richland,IA,41.194129,-91.98027,-6,1,"41.194129, -91.98027",41.194129,-91.98027


In [10]:
# drop the unecessary columns that are not needed
df_new = df_new.drop(['latitude', 'longitude', 'Daylight savings time flag', 'Timezone'], axis=1)
df_new.head()

,Zip,City,State,Latitude,Longitude,geopoint
0,71937,Cove,AR,34.398483,-94.39398,"34.398483, -94.39398"
1,72044,Edgemont,AR,35.624351,-92.16056,"35.624351, -92.16056"
2,56171,Sherburn,MN,43.660847,-94.74357,"43.660847, -94.74357"
3,49430,Lamont,MI,43.010337,-85.89754,"43.010337, -85.89754"
4,52585,Richland,IA,41.194129,-91.98027,"41.194129, -91.98027"


In [11]:
# drop once again and create a new column
df_gcd = df_new.drop(['Zip','geopoint'], axis=1)
df_gcd.head()

,City,State,Latitude,Longitude
0,Cove,AR,34.398483,-94.39398
1,Edgemont,AR,35.624351,-92.16056
2,Sherburn,MN,43.660847,-94.74357
3,Lamont,MI,43.010337,-85.89754
4,Richland,IA,41.194129,-91.98027


In [12]:
# map df_gcd to df using City and State creating a new dataframe called df_result
df_result = pd.merge(df, df_gcd, on=['City','State'], how='left')
df_result.head()

,Unnamed: 0,Unnamed: 0.1,MinistryMonthlyBudgetAmount,StartDate,EndDate,Marital_Status,Number_of_Household_Members,City,State,PostalCode,Country,Coded_Marital_Status,End_Yr,Start_Yr,End_Mnth,Start_Mnth,Address,Latitude,Longitude
0,0,1,5485,3/1/2013,12/31/2013,Married,2,Erlanger,KY,41018,United States of America,2.0,2013.0,2013.0,12.0,3.0,"Erlanger, KY, United States of America",39.013755,-84.602290
1,1,2,4454,11/1/2011,4/30/2012,Married,2,Spanish Fort,AL,36527,United States of America,2.0,2012.0,2011.0,4.0,11.0,"Spanish Fort, AL, United States of America",30.668757,-87.939710
2,1,2,4454,11/1/2011,4/30/2012,Married,2,Spanish Fort,AL,36527,United States of America,2.0,2012.0,2011.0,4.0,11.0,"Spanish Fort, AL, United States of America",30.684873,-87.897230
3,2,3,767,1/1/2013,12/31/2013,Single,1,Lexington,KY,40517,United States of America,4.0,2013.0,2013.0,12.0,1.0,"Lexington, KY, United States of America",38.037847,-84.616450
4,2,3,767,1/1/2013,12/31/2013,Single,1,Lexington,KY,40517,United States of America,4.0,2013.0,2013.0,12.0,1.0,"Lexington, KY, United States of America",38.028269,-84.471505


In [13]:
# to many duplicates drop the extra records with the command below
df_result.shape

(147864, 19)

In [14]:
df_result.drop_duplicates(subset = ['MinistryMonthlyBudgetAmount', 'StartDate',
                                    'EndDate', 'Marital_Status',
                                    'Number_of_Household_Members','City',
                                    'State','PostalCode','PostalCode','Country',
                                    'Coded_Marital_Status', 'End_Yr', 'Start_Yr',
                                    'Start_Mnth'], inplace=True)
df_result.shape

(8893, 19)

In [15]:
df.shape

(8900, 17)

In [16]:
df_result.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'MinistryMonthlyBudgetAmount',
       'StartDate', 'EndDate', 'Marital_Status', 'Number_of_Household_Members',
       'City', 'State', 'PostalCode', 'Country', 'Coded_Marital_Status',
       'End_Yr', 'Start_Yr', 'End_Mnth', 'Start_Mnth', 'Address', 'Latitude',
       'Longitude'],
      dtype='object')

In [17]:
# df_result.Latitude = df_result.Latitude.round(decimals=2)
# df_result.Longitude = df_result.Longitude.round(decimals=2)
# df_result.head()

# to many null values
df_result.isnull().sum()

Unnamed: 0                       0
Unnamed: 0.1                     0
MinistryMonthlyBudgetAmount      0
StartDate                        0
EndDate                          0
Marital_Status                   0
Number_of_Household_Members      0
City                             0
State                            0
PostalCode                       0
Country                          0
Coded_Marital_Status            10
End_Yr                           1
Start_Yr                         1
End_Mnth                         1
Start_Mnth                       1
Address                          0
Latitude                       384
Longitude                      384
dtype: int64

In [18]:
# back up dataset where null values are dropped
df_result_dropped = df_result.dropna()
df_result_dropped.isnull().sum()

Unnamed: 0                     0
Unnamed: 0.1                   0
MinistryMonthlyBudgetAmount    0
StartDate                      0
EndDate                        0
Marital_Status                 0
Number_of_Household_Members    0
City                           0
State                          0
PostalCode                     0
Country                        0
Coded_Marital_Status           0
End_Yr                         0
Start_Yr                       0
End_Mnth                       0
Start_Mnth                     0
Address                        0
Latitude                       0
Longitude                      0
dtype: int64

In [19]:
# create a masked dataset of all the null values for latitude/longitude
# no_lat = df_result['Latitude'].isnull()
# no_lat.head()

In [20]:
#put it back into a new dataframe and we have all of the latitudes and longitudes that are null 
# df_nulls = df_result[no_lat]
# df_nulls.shape

In [21]:
# next we will work on geocoding a portion of the data
# data = df_nulls['Address'][:4]
# geo = geocode(data, provider='nominatim', user_agent='csc_user_ht')
# geo

In [22]:
# implement this with itertools so we can chunk the data into parts instead of doing it all at once
# for chunking the dataset work on splitting the 384 records into groups of 10 progressing each chunk
# by 10 until we reach the max number of records from the data set
# concatenate each lat and long onto a new dataframe 
# concatenate that new dataframe to df_result as the new lats and longs
# drop the old lats and longs

# records_index = np.arange(0,385)
# sliced = itertools.islice(records_index, 10)
# for i in sliced:
#     print(i)

In [23]:
# addresses = df_nulls['Address'][:100]
# addresses

In [24]:
# for i in range(0,100,10):
#     print(i)

In [25]:
# addresses = df_nulls['Address'][:100]
# df_geocoded = pd.DataFrame()
# for i in range(0,110,10):
#     start = i
#     end = i + 10
#     addresses=addresses[start:end]
#     geo = geocode(data, provider='nominatim', user_agent='csc_user_ht')
#     frames = [df_geocoded, geo] 
#     df_geocoded = pd.concat(frames)

# df_geocoded

In [26]:
# df_geocoded.drop_duplicates()

In [27]:
# addresses = df_nulls['Address'][100:200]

# for i in range(50):
#     start = i
#     end = i + 10
#     addresses[start:end]
#     geo = geocode(data, provider='nominatim', user_agent='csc_user_ht')
#     frames = [df_geocoded, geo] 
#     df_geocoded = pd.concat(frames)

# df_geocoded

In [28]:
df_result_dropped.head()

,Unnamed: 0,Unnamed: 0.1,MinistryMonthlyBudgetAmount,StartDate,EndDate,Marital_Status,Number_of_Household_Members,City,State,PostalCode,Country,Coded_Marital_Status,End_Yr,Start_Yr,End_Mnth,Start_Mnth,Address,Latitude,Longitude
0,0,1,5485,3/1/2013,12/31/2013,Married,2,Erlanger,KY,41018,United States of America,2.0,2013.0,2013.0,12.0,3.0,"Erlanger, KY, United States of America",39.013755,-84.602290
1,1,2,4454,11/1/2011,4/30/2012,Married,2,Spanish Fort,AL,36527,United States of America,2.0,2012.0,2011.0,4.0,11.0,"Spanish Fort, AL, United States of America",30.668757,-87.939710
3,2,3,767,1/1/2013,12/31/2013,Single,1,Lexington,KY,40517,United States of America,4.0,2013.0,2013.0,12.0,1.0,"Lexington, KY, United States of America",38.037847,-84.616450
53,3,4,6368,7/1/2015,2/29/2016,Married,2,Manheim,PA,17545,United States of America,2.0,2016.0,2015.0,2.0,7.0,"Manheim, PA, United States of America",40.168294,-76.412300
54,4,5,1919,8/1/2011,7/31/2012,Divorced,1,Jacksonville,FL,32258-5434,United States of America,1.0,2012.0,2011.0,7.0,8.0,"Jacksonville, FL, United States of America",30.344940,-81.683107


In [29]:
df_result_dropped = df_result_dropped.drop(columns=['Unnamed: 0.1'])

In [30]:
df_result_dropped['Mnths_Passed'] = ((df_result_dropped['End_Yr'] * 12) + df_result_dropped['End_Mnth']) - ((df_result_dropped['Start_Yr'] * 12) + df_result_dropped['Start_Mnth']) 
df_result_dropped['Mnths_Passed']

0          9.0
1          5.0
3         11.0
53         7.0
54        11.0
          ... 
147856    13.0
147857    11.0
147858     6.0
147859    11.0
147862    10.0
Name: Mnths_Passed, Length: 8498, dtype: float64

In [31]:
df_result_dropped.columns=['Index', 'MinistryMonthlyBudgetAmount', 'StartDate', 'EndDate',
       'Marital_Status', 'Number_of_Household_Members', 'City', 'State',
       'PostalCode', 'Country', 'Coded_Marital_Status', 'End_Yr', 'Start_Yr',
       'End_Mnth', 'Start_Mnth', 'Address', 'Latitude', 'Longitude',
       'Mnths_Passed']
# df_result_dropped.head()
df_result_dropped.tail()

,Index,MinistryMonthlyBudgetAmount,StartDate,EndDate,Marital_Status,Number_of_Household_Members,City,State,PostalCode,Country,Coded_Marital_Status,End_Yr,Start_Yr,End_Mnth,Start_Mnth,Address,Latitude,Longitude,Mnths_Passed
147856,8894,6339,1/1/2012,2/28/2013,Married,2,Erlanger,KY,41018,United States of America,2.0,2013.0,2012.0,2.0,1.0,"Erlanger, KY, United States of America",39.013755,-84.602290,13.0
147857,8895,5775,1/1/2015,12/31/2015,Married,2,Waxhaw,NC,28173,United States of America,2.0,2015.0,2015.0,12.0,1.0,"Waxhaw, NC, United States of America",34.929433,-80.730610,11.0
147858,8896,5939,12/1/2009,6/30/2010,Married,2,Bowman,ND,58623-0122,United States of America,2.0,2010.0,2009.0,6.0,12.0,"Bowman, ND, United States of America",46.182799,-103.404530,6.0
147859,8897,6515,9/1/2005,8/31/2006,Married,2,Duncanville,TX,75116,United States of America,2.0,2006.0,2005.0,8.0,9.0,"Duncanville, TX, United States of America",32.767268,-96.777626,11.0
147862,8898,4079,7/1/2009,5/31/2010,Married,2,Lake Forest,CA,92630,United States of America,2.0,2010.0,2009.0,5.0,7.0,"Lake Forest, CA, United States of America",33.640223,-117.691080,10.0


In [32]:
df_rd_mini = df_result_dropped.copy()
df_rd_mini.tail()

,Index,MinistryMonthlyBudgetAmount,StartDate,EndDate,Marital_Status,Number_of_Household_Members,City,State,PostalCode,Country,Coded_Marital_Status,End_Yr,Start_Yr,End_Mnth,Start_Mnth,Address,Latitude,Longitude,Mnths_Passed
147856,8894,6339,1/1/2012,2/28/2013,Married,2,Erlanger,KY,41018,United States of America,2.0,2013.0,2012.0,2.0,1.0,"Erlanger, KY, United States of America",39.013755,-84.602290,13.0
147857,8895,5775,1/1/2015,12/31/2015,Married,2,Waxhaw,NC,28173,United States of America,2.0,2015.0,2015.0,12.0,1.0,"Waxhaw, NC, United States of America",34.929433,-80.730610,11.0
147858,8896,5939,12/1/2009,6/30/2010,Married,2,Bowman,ND,58623-0122,United States of America,2.0,2010.0,2009.0,6.0,12.0,"Bowman, ND, United States of America",46.182799,-103.404530,6.0
147859,8897,6515,9/1/2005,8/31/2006,Married,2,Duncanville,TX,75116,United States of America,2.0,2006.0,2005.0,8.0,9.0,"Duncanville, TX, United States of America",32.767268,-96.777626,11.0
147862,8898,4079,7/1/2009,5/31/2010,Married,2,Lake Forest,CA,92630,United States of America,2.0,2010.0,2009.0,5.0,7.0,"Lake Forest, CA, United States of America",33.640223,-117.691080,10.0


In [33]:
df_rd_mini = df_rd_mini.drop(columns=['End_Yr', 'Start_Yr', 'End_Mnth', 'Start_Mnth'], axis=1)

In [34]:
df_rd_mini['StartDate'] =  pd.to_datetime(df_rd_mini['StartDate'])
df_rd_mini['EndDate'] =  pd.to_datetime(df_rd_mini['EndDate'])

In [35]:
df_rd_mini.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8498 entries, 0 to 147862
Data columns (total 15 columns):
Index                          8498 non-null int64
MinistryMonthlyBudgetAmount    8498 non-null int64
StartDate                      8498 non-null datetime64[ns]
EndDate                        8498 non-null datetime64[ns]
Marital_Status                 8498 non-null object
Number_of_Household_Members    8498 non-null int64
City                           8498 non-null object
State                          8498 non-null object
PostalCode                     8498 non-null object
Country                        8498 non-null object
Coded_Marital_Status           8498 non-null float64
Address                        8498 non-null object
Latitude                       8498 non-null float64
Longitude                      8498 non-null float64
Mnths_Passed                   8498 non-null float64
dtypes: datetime64[ns](2), float64(4), int64(3), object(6)
memory usage: 1.0+ MB


In [36]:
df_rd_mini.shape

(8498, 15)

In [37]:
df_result_dropped.shape

(8498, 19)

In [38]:
# df_result.to_csv('../../Data/OutsideData/formatted_lat_long_for_Kmeans.csv')
df_result_dropped.to_csv('../../Data/OutsideData/formatted_lat_long_for_Kmeans.csv')

In [39]:
df_rd_mini.to_csv('../../Data/OutsideData/datestimes_for_Kmeans.csv')

In [40]:
df_all_num = df_rd_mini.copy()

In [41]:
df_all_num.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8498 entries, 0 to 147862
Data columns (total 15 columns):
Index                          8498 non-null int64
MinistryMonthlyBudgetAmount    8498 non-null int64
StartDate                      8498 non-null datetime64[ns]
EndDate                        8498 non-null datetime64[ns]
Marital_Status                 8498 non-null object
Number_of_Household_Members    8498 non-null int64
City                           8498 non-null object
State                          8498 non-null object
PostalCode                     8498 non-null object
Country                        8498 non-null object
Coded_Marital_Status           8498 non-null float64
Address                        8498 non-null object
Latitude                       8498 non-null float64
Longitude                      8498 non-null float64
Mnths_Passed                   8498 non-null float64
dtypes: datetime64[ns](2), float64(4), int64(3), object(6)
memory usage: 1.0+ MB


In [42]:
df_all_num = df_all_num.drop(columns=['City', 'Country'])

In [43]:
df_all_num.to_csv('../../Data/OutsideData/smaller_data_for_Kmeans.csv')